# DATA 550 Lab 4 - Storytelling and visualizing trends and uncertainties

# Group 3: Rachel Kamphuis, Tao Wang¶

There are two versions of this lab, one in Python and one in R.
The R lab will use `ggplot` and the Python lab will use `Altair`.

This is the Python version.

Each partner should choose a version to complete, though keep in mind that **you are required to alternate between completing the R labs and the Python labs** to get experience using both languages.

## Acknowledgemnts

This lab was lightly adapted from the one created by Joel Ostblom for the DSCI 531 version of Data Viz I in the 2020/21 Academic year.

<div class="alert alert-info" style="color:black">

## Submission instructions
rubric={mechanics:2}

<p>You receive marks for submitting your lab correctly, please follow these instructions:</p>

<ul>
  <li>Each partner should complete one file, though collaboration is permitted.</li>
  <li>Push your <code>.ipynb</code> files to GitHub frequently (i.e. not just in one fell swoop at the end).</li>
  <li>Don't forget to submit a clickable link to your GitHub repo on Canvas so we can grade it.</li>
</ul>
</div>

# 0. The back story

*(If you don't like the back stories, you can jump down to the heading "The data" Also a disclaimer that I am not a linguist, so this lab is my understanding of some articles I have read, it is likely not perfect.)*

First, I just want to thank you for bringing me back to my senses and stopping my world domination attempt and silly startup idea. I don't know what happen, but I clearly wasn't myself there for a while. Now that you have brought me back to normal, I would like design this lab more similar to the first one and go through a topic that I find intriguing and ponder-worthy.

This time, instead of looking around us, we will look inside. Ever since I arrived on this planet, I have been fascinated about your use of airborn oscillations to transmit information generated by electrical impulses. And the fact that you rely on muscular dexterity for the encoding process as well as passive mechnotransduction for the decoding is truly remarkable (in a primitive sense of course, I guess how your kind can appreciate "the remarkable complexity" of a cave painting). Such a system is clearly a bottleneck in terms of information bandwidth and I am surprised a more direct means of intercranial transmission never evolved here, but from the perspective of my research on less evolved galactic lifeforms I find this fascinating!

After hearing that there are over 7,000 languages spoken across this globe, I am curious whether some of these at least allow for more rapid information transmission than others. The implications of this reach beyond just communication as I have recently learned that you also restrict many of your thoughts to those that can be expressed in words (again, fascinating!). This means that you could literally upgrade your thought throughput by learning a new language. And it could mean that there are groups of people who have access to faster thinking others, just because their language can convey information at a higher rate, what an enormous advantage!

## The data

To aid our exploration of whether some languages are more efficient at conveying information than others, I have received assistance from some of your finest scientists. The data in this lab was published as part of a study in Science Advances in 2019 and made publicly available in their online repository. If you want to read part of the article, I recommend that you do it after attempting to answer the questions (or if you get stuck) since it will give away some of the things that we can otherwise "discover" ourselves. There is also a popular science version of this article publish in The Economist, and I link to both in the optional question at the end.

The scientists have compiled two tables, one with general information on the languages we are studying and one with experimental data where they recorded people speaking a certain text in different languages and noted down how fast they spoke, etc. You can find a description of the columns of both datasets below and they are available in your lab repo.

---

<center><h3>Languages dataset</h4></center>

| Column              | Description                                       |
|---------------------|---------------------------------------------------|
| iso_lang            | ISO_639-3 language code                           |
| language            | Language name                                     |
| information density | Bits of information per syllable in the language  |
| distinct_syllables  | The number of different syllables in the language |
| continent           | The continent where the language is spoken        |

---

<center><h3>Spoken texts dataset</h4></center>

| Column    | Description                                     |
|-----------|-------------------------------------------------|
| speaker   | Speaker ID                                      |
| iso_lang  | ISO_639-3 language code                         |
| text      | Text ID                                         |
| sex       | The sex of the speaker                          |
| duration  | The number of seconds it took to speak the text |
| syllables | Number of syllables uttered during the speech   |
| age       | The age of the speaker                          |

---

Overall, you will have a bit less direction in this lab to closer simulate a real-life scenario. The questions are a mix of explicit instructions and describing the problem in text, rather than in exact steps. As part of this, there is also a few data wrangling steps needed. As this is an exercise in you interpreting information, it is up to you to clarify what you don't understand with me and the TAs during labs and office hours, just as you would in a real-life collaboration.

In [3]:
# Run this cell to ensure that altair plots show up in the exported HTML
# and that the R cell magic works
import altair as alt

# Save a vega-lite spec and a PNG blob for each plot in the notebook
# alt.renderers.enable('mimetype')
# Handle large data sets without embedding them in the notebook
# alt.data_transformers.enable('data_server')

alt.renderers.enable("html")

alt.data_transformers.enable("default")




DataTransformerRegistry.enable('default')

# 1. Language exploration

<p>Let’s first get familiar with potential differences and similarities between the languages in this data set. Clearly a language's information density is going to be important in how quick a speaker can convey information, but I wonder if number of distinct syllabus matter. Let's see if we can see any patterns by organizing the languages by these two variables.</p>

<div class="alert alert-info" style="color:black">

### Question 1
rubric={accuracy:1,visualization:1}

<h4>Python</h4>
<ol type="1">
<li>Create a scatterplot of the information density of each language versus its number of distinct syllabus. Zoom in to the extent of the data instead of forcing zero to be in the axis.</li>
<li>You will notice that the data appears to cluster in groups in the scatter plot. Can you find a categorical variable in this dataframe that roughly explains this clustering? Color the points in the plot you just created according that variable and add a suitable trend line for each cluster. If there are any points that seem to be outliers, avoid overfitting the trendline to them.</li>
<li>If you think any particular data points deviate from their cluster, label them with their language name in the plot.</li>
</ol>

</div>

In [30]:
import pandas as pd

languages = pd.read_csv("languages.csv")

# base scatter plot: distinct syllables on x, information density on y
base = alt.Chart(languages).encode(
    x=alt.X(
        "distinct_syllables:Q",
        title="Distinct syllables",
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y(
        "information_density:Q",
        title="Information density (bits per syllable)",
        scale=alt.Scale(zero=False)
    ),
    color=alt.Color("continent:N", title="Continent"),
    tooltip=["language", "continent", "distinct_syllables", "information_density"]
)

points = base.mark_circle(size=60, opacity=0.8)

# one regression line per continent
trend = base.transform_regression(
    "distinct_syllables",
    "information_density",
    groupby=["continent"]
).mark_line(size=2)

# label outlier
outlier_langs = ["English", "Vietnamese","French"]

labels = (
    alt.Chart(languages[languages["language"].isin(outlier_langs)])
    .mark_text(align="left", dx=5, dy=-5)
    .encode(
        x="distinct_syllables:Q",
        y="information_density:Q",
        text="language"
    )
)

plot_A = (points + trend + labels).properties(
    width=500,
    height=350,
    title="Information density vs. distinct syllables"
)

plot_A



alt.LayerChart(...)

# 2. Speech rate and information density

Hmm... what an intriguing relationship we just revealed... let’s keep these clusters in mind while exploring whether continuing our exploration. The two variables that directly determine the information rate of a language are its information density (bits of information per syllable) and the rate at which it is spoken (syllables per second). A high value in both would indicate a high information rate and efficient communication (a higher number of information bits conveyed per second).

<div class="alert alert-info" style="color:black">

### Question 2
rubric={reasoning:2,accuracy:3}
    
<h4>Python</h4> 
<ol type="1">
<li>While the information density of each language have already been provided to us, we will have to approximate the speech rate (syllables per second) of each language via the <code>spoken_texts</code> dataset. Using this data, calculate the average speech rate per language (for now, treat all entries as individual observations, even if some are read by the same person).</li>
    <li>Create a new column, <code>speech_rate</code>, in the <code>languages</code> data frame that hold these average values. Since the calculated values are not in the same order as the languages in the <code>languages</code> data frame, you cannot simply append it. Instead, you can use the dataframe merge method to join the series of average speech rate values that you just created with the <code>languages</code> dataframe. The language ISO codes could be used as the column to merge on.</li>
<li>Create a scatter plot of the information density versus the speech rate for the <code>languages</code> dataframe. Use the same groupings as in the previous scatter plot you made and add one trend line per group. Label the same points with text labels as in the previous scatterplot. Zoom in to the extent of the data instead of forcing zero to be in the axes.</li>
<li>Write a few sentences under the plot describing the relationship you observe. Based on this plot, do speakers of any particular languages appear to be more efficient at conveying information?</li>
</ol>
    
</div>

In [28]:


# load the data 
languages = pd.read_csv("languages.csv")
spoken_texts = pd.read_csv("spoken-texts.csv")

# compute average speech rate per language 

# speech rate for each recording: syllables per second
spoken_texts["speech_rate"] = spoken_texts["syllables"] / spoken_texts["duration"]

avg_speech_rate = (
    spoken_texts
    .groupby("iso_lang", as_index=False)["speech_rate"]
    .mean()
)

# merge back into languages dataframe
languages = languages.merge(avg_speech_rate, on="iso_lang", how="left")

# make a scatter plot: information density (x) vs speech rate (y)

base = alt.Chart(languages).encode(
    x=alt.X(
        "information_density:Q",
        title="Information density (bits per syllable)",
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y(
        "speech_rate:Q",
        title="Average speech rate (syllables per second)",
        scale=alt.Scale(zero=False)
    ),
    color=alt.Color("continent:N", title="Continent"),
    tooltip=["language", "continent", "information_density", "speech_rate"]
)

# points
points = base.mark_circle(size=60, opacity=0.8)

# regression line per continent
trend = alt.Chart(languages).transform_regression(
    "information_density",
    "speech_rate",
    groupby=["continent"]
).mark_line(size=2).encode(
    x="information_density:Q",
    y="speech_rate:Q",
    color=alt.Color("continent:N")
)

# label a couple of interesting languages
outlier_langs = ["English", "Vietnamese","French"]

labels = (
    alt.Chart(languages[languages["language"].isin(outlier_langs)])
    .mark_text(align="left", dx=5, dy=-5)
    .encode(
        x="information_density:Q",
        y="speech_rate:Q",
        text="language"
    )
)

plot_B = (points + trend + labels).properties(
    width=500,
    height=350,
    title="Information density vs. speech rate"
)

plot_B




alt.LayerChart(...)

### Answer

From this plot we see a clear negative relationship between information density (x-axis) and speech rate (y-axis). Languages that pack more bits into each syllable tend to be spoken more slowly, so their points are further to the right but lower on the plot. Languages with lower information density are spoken more quickly, so they appear more to the left and higher up. Asian languages mostly sit from the top-left to the middle area, while many European languages stretch from the centre towards the bottom-right. Vietnamese has very high information density but a relatively low speech rate, French has higher density than most other European languages, and English sits above the main European cluster with medium density and medium speed. Overall, the pattern suggests a trade-off: as information density increases, speech rate decreases.

Based on this plot alone, I do not think speakers of any particular language look clearly more efficient at conveying information. If we think of “efficiency” as information per second (information density × speech rate), the negative relationship in the figure suggests that faster languages tend to carry less information per syllable, and denser languages tend to be spoken more slowly. In other words, gains in one dimension are compensated by losses in the other. This makes it unlikely that one language dominates both dimensions

# 3. Language information rate

Oh those results are very interesting… Hmmm I wonder why it looks like that… Ahhh I see what happened, I forgot to include the severe limitations of the feeble human mind in my hypothesis, oh oops you’re still here, ehh well 🪄*obliviate* to you and let’s carry on.

<div class="alert alert-info" style="color:black">

### Question 3
rubric={accuracy:3,viz:2}

<h4>Python</h4>
<ol type="1">
<li>It looks like we do not have a language that is high both in information density and speech rate, which could indicate that human minds are not good at processing auditory information beyond a certain rate limit (read more in the article if you're interested). Let’s directly plot the rate at which information is conveyed for all the languages to see if there are at least small differences between them. Combine the appropriate columns to create a new column, <code>information_rate</code>, which holds the amount of information per second conveyed by each speaker.</li>
<li>Create a plot that has the language ISO codes on one axis and the information rate on the other. Plot a graphical representation of either the distribution of data or every single data point. Color this by the same variable as in the previous two scatter plots. Sort the languages in a suitable order.</li>
<li>Include a mark for the mean and the 95% confidence interval on top of the marks you just created above. Color this in a suitable way so that it stands out from the distribution/individual observations created in the previous point.</li>
</ol>
</div>

In [31]:


# bring information_density (and continent) onto each spoken_texts row 

info_cols = ["iso_lang", "language", "information_density", "continent"]
spoken_with_info = spoken_texts.merge(
    languages[info_cols],
    on="iso_lang",
    how="left"
)

# compute information_rate (bits per second) for each speaker 

spoken_with_info["information_rate"] = (
    spoken_with_info["speech_rate"] * spoken_with_info["information_density"]
)

# decide the ordering of languages: sort by mean information_rate 

lang_order = (
    spoken_with_info
    .groupby("iso_lang")["information_rate"]
    .mean()
    .sort_values(ascending=False)        # highest information rate at the top
    .index
    .tolist()
)

# make a plot information rate by language (ISO code)

base = alt.Chart(spoken_with_info).encode(
    x=alt.X(
        "information_rate:Q",
        title="Information rate (bits per second)",
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y(
        "iso_lang:N",
        title="Language (ISO 639-3 code)",
        sort=lang_order
    ),
    # Same grouping variable as in previous questions: continent
    color=alt.Color("continent:N", title="Continent"),
    tooltip=["language", "iso_lang", "continent", "information_rate"]
)

# individual observations (one point per speaker)
points = base.mark_circle(size=50, opacity=0.7)

# 95% confidence interval for each language
ci_band = alt.Chart(spoken_with_info).mark_errorbar(
    extent="ci",
    color="black"
).encode(
    x="information_rate:Q",
    y=alt.Y("iso_lang:N", sort=lang_order)
)

# mean information rate per language
means = alt.Chart(spoken_with_info).mark_point(
    size=80,
    filled=True,
    color="black"
).encode(
    x="mean(information_rate):Q",
    y=alt.Y("iso_lang:N", sort=lang_order)
)

plot_C = (points + ci_band + means).properties(
    width=600,
    height=350,
    title="Information rate by language"
)

plot_C

alt.LayerChart(...)

# 4. Tell a story

<div class="alert alert-info" style="color:black">

### Question 4
rubric={reasoning:3}
    
These results are looking pretty promising, now we just need to add a narrative so that it will be easy for others to follow along with what we have done here. Tell a story using the three plots we have created. Build it up according to the principles and examples from this week's required readings. In your story you can refer to the plots as plot A,B, and C. You story should make it clear what the plot is telling us and how it motivates the next plot we're making so that there is a clear story line for the reader to follow as well as a clear take home message (there might be many messages in these plots, so you can choose which to focus on). No need to write an essay here, a succinct paragraph per plot should do it.

</div>

### Answer
Plot A – Information density vs. distinct syllables
Plot A shows the basic structure of the languages. The x-axis is the number of distinct syllables and the y-axis is information density (bits per syllable). We can see two loose groups. Asian languages are on the left with fewer syllables but higher information density. European languages are more to the right with more syllables and a bit lower density. Vietnamese has very high information density for its number of syllables, so it stands out at the top of the Asian group. French has higher density than most other European languages. English has the largest number of syllables and is far to the right. No language is extreme on both axes at the same time.

Plot B – Information density vs. speech rate
Plot B looks at how people actually speak. The x-axis is still information density, and the y-axis is speech rate in syllables per second. In both continents we see a clear negative pattern: languages with high information density are spoken more slowly, and languages with low density are spoken more quickly. Vietnamese is very dense but has only a moderate speech rate. Some European languages, like Spanish, are spoken quickly but have lower density. French has higher density and a middle speech rate, and English is also in the middle range for both. This plot tells us there is a trade-off. If each syllable carries more information, speakers slow down; if each syllable carries less, speakers speed up.

Plot C – Information rate by language
Plot C answers that question. It shows information rate (bits per second) for each language. Each small point is one speaker. The black dots are the mean, and the black lines are 95% confidence intervals. Some languages such as French, English and Vietnamese have slightly higher means. Some others are slightly lower. But the means all fall in a fairly narrow band, and the confidence intervals overlap a lot. This suggests that the trade-off from Plot B cancels out in the end. Fast but less dense languages and slower but denser languages end up sending information at almost similar rates.

Therefore, all these three plots tell us that human languages use different coding strategies: some have many syllables, some have few; some are spoken fast, some slow. But because of the trade-off between density and speed, the final information rate per second is quite similar across languages. It looks like human speech and hearing place a limit on how much information we can handle, and all languages stay close to that limit.



# 5. Figure composition

<p>Now that we have the individual figures and the story, the final step for us here is to compile our figures into the same layout for presentation.</p>

<div class="alert alert-info" style="color:black">

### Question 5
rubric={accuracy:1,viz:2}
    
    
<h4>Python</h4>
<ol type="1">
<li>Assign each of your previous three plots to a variable name (you don’t have to repeat the code down here, you can assign it in the original cell, just make sure you are still outputting the variable at the end of those cells so that the plots still are displayed after each cell).</li>
<li>Arrange the three plots into a layout with the two scatter plots in a column to the left and mean + CI plot to the right. The overall layout should look even, so make the scatterplots about half the height and two thirds of the width of the mean + CI plot.</li>
<li>Set proper titles where you judge it is needed. Show that you know the proper command to save this as a png file on your local computer.</li>
</ol>
    
</div> 

In [32]:
# adjust sizes so that the two scatterplots are
# about half the height and ~2/3 of the width of the mean+CI plot
plot_A_panel = plot_A.properties(
    width=300,
    height=250,
    title="A. Information density vs. distinct syllables"
)

plot_B_panel = plot_B.properties(
    width=300,
    height=250,
    title="B. Information density vs. speech rate"
)

plot_C_panel = plot_C.properties(
    width=450,
    height=500,
    title="C. Information rate by language"
)

# arrange A and B in a column on the left, C on the right
panel_figure = (plot_A_panel & plot_B_panel) | plot_C_panel

panel_figure


alt.HConcatChart(...)

In [33]:
panel_figure.save("language_information_panel.png")

---

Thank you, this has certainly been educational. I'll present these results to the intergalactic ministry of traffic and hopefully we can spare earth from the hyperspace bypass this time around... ah shoot, me and my big mouth! Basically there is nothing to worry about, you just keep squabbling amongst each other over lines in the sand and hopefully you wont hear from us again. So long and thanks for all the fish! 🛸🐬

---

# 6. Presenting figures to a general or a technical audience 

<p>Below you can see the original figure from <a href="https://www.science.org/doi/10.1126/sciadv.aaw2594">the scientific article in “Science Advances”</a> and the <a href="https://www.economist.com/graphic-detail/2019/09/28/why-are-some-languages-spoken-faster-than-others">simplified version that was published in the Economist</a> (you can create a free account to read the article, no need to pay or use the trial). There is also an R Markdown file for how they did their analysis in the article supplementary materials in case you're interested.</p>

<div class="alert alert-warning" style="color:black">

### Question 6 (Optional)
rubric={reasoning:1.5}

<ol type="1">
<li>Briefly describe how the Economist have simplified the figure from the original scientific publication.</li>
<li>Do you agree with choices made by the Economist? Is there anything you think could have been done differently or additionally, that would have communicated the findings more clearly to a general / popular science audience? This could be based on design choices or your understanding of the data.</li>
</ol>

</div>

## Scientific figure

![image.png](article.png)

## Popular science figure

![image.png](economist.png)

### Answer: 

The original scientific figure is quite complex. It has two panels, one for speech rate (SR) and one for information rate (IR). Each panel shows one row per language plus an extra row for “All”. For every language we see a smooth density curve, many small vertical ticks for each individual speaker, several vertical dashed lines, and different colours for different language families. This gives a lot of information but it also makes the plot visually heavy and harder to read for someone who is not used to scientific graphics.

The Economist version removes most of this extra detail. It keeps the basic layout of two panels, but now each panel uses just one colour. The “All” row disappears and there is no legend for language family. The small ticks for individual speakers are gone, so we only see the smooth shapes. There are far fewer grid lines and dashed lines. Instead, the main guidance for the reader comes from two simple arrows and labels: “Speak more quickly →” and “Convey information more quickly →”. As a result, the reader’s attention goes straight to the shapes and to the direction of the arrows, which makes the main idea much easier to see.

Overall I agree with most of the choices made by The Economist. For a general audience, the goal is to communicate the key message, not to show every technical detail. Using one colour per panel, removing the ticks, and dropping the family legend all help non-experts understand the story without feeling overwhelmed. If I changed anything, I might add a short subtitle in plain language, such as “Different languages, similar overall information rate”, to state the conclusion directly. I might also keep one very light vertical line for the overall mean in each panel, so readers have a clear reference point. But in general, their simplified figure is a good example of how to turn a complex scientific plot into something that works well in a popular science article.

# 7. Tell your story

<p>Only do this exercise if you feel like you have time over after completing all labs and want additional feedback on your story telling. It will likely take some time and does not give that many points.</p>

<div class="alert alert-warning" style="color:black">

### Question 7 (Optional)
rubric={reasoning:1}
 
<p>Use the same data to tell another story than the one we told here. Suggestively, you would do some EDA and then use around 2-3 figures with accompanying text to tell a short story about an interesting relationship that you found. I encourage you to try to think of an idea of your own, but if you can't come up with one, I have listed a few ideas below.</p>
    
<details><summary>Suggested ideas to explore</summary>
<ul>
<li>Does the particular text matter for which continent or language was faster? (you could do a paired comparison here)</li>
<li>Group by each speaker’s mean instead of showing all texts (I would personally prefer this to what we did above as I think it shows the variation and sample size more truly, although the means will be similar)</li>
<li>Add language families from the paper and see if there are any patterns among them.</li>
<li>Add additional language features such as tonality and see if there any patterns there.</li>
<li>Plot the different language features on a map.</li>
</ul>
</details>

</div>

In [ ]:
# YOUR ANSWER GOES HERE

YOUR ANSWER GOES HERE

---

A couple of interesting videos on communication and language if you have some spare time this weekend:

- Switching from auditory transfer to electric via implants such as [Kevin Warvick's cyborg-like connection of his nervous system to the internet and his wife's jewelry](https://www.youtube.com/watch?v=Z8HeFNJjuj0&feature=youtu.be&t=822).
- There is evidence that the words we have at our disposal influences how we think. This could mean that certain thought patterns and emotions might be more accessible in languages where they are more easily verbalized, and maybe we would have slightly different thoughts if we knew another language (the reference here is the weak form of the Sapiro-Whorf hypothesis of linguistic relativity, not the strong version). After you are done with the lab, I recommend listening to [this talk by Lera Boroditsky](https://www.youtube.com/watch?v=RKK7wGAYP6k) or watching the movie "Arrival" (although they go a bit far...) if you are interested in learning more about this (it's fun to think about!).

---

# Submission to Canvas

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Run All Cells...`
2. Commit and push all your work (including the .ipynb file) to GitHub.
3. Submit a link to your repo on Canvas (this has to be done only once per group).

---

# Help us improve the labs

The MDS program is continually looking to improve our courses, including lab questions and content. The following optional questions will not affect your grade in any way nor will they be used for anything other than program improvement:

1. Approximately how many hours did you spend working or thinking about this assignment (including lab time)?

#Ans:

2. Were there any questions that you particularly liked or disliked?

#Ans: [Questions you liked]

#Ans: [Questions you disliked]